# Import Library

In [1]:
import numpy as np
import cv2
from skimage.transform import resize
from skimage import feature 
import matplotlib.pyplot as plt
from PIL import Image
from pathlib import Path
import os
import re

In [2]:
def get_pixel(img, center, x, y): 
    new_value = 0
      
    try: 
        # If local neighbourhood pixel  
        # value is greater than or equal 
        # to center pixel values then  
        # set it to 1 
        if img[x][y] >= center: 
            new_value = 1
              
    except: 
        # Exception is required when  
        # neighbourhood value of a center 
        # pixel value is null i.e. values 
        # present at boundaries. 
        pass
      
    return new_value 
   
# Function for calculating LBP 
def lbp_calculated_pixel(img, x, y): 
   
    center = img[x][y] 
   
    val_ar = [] 
      
    # top_left 
    val_ar.append(get_pixel(img, center, x-1, y-1)) 
      
    # top 
    val_ar.append(get_pixel(img, center, x-1, y)) 
      
    # top_right 
    val_ar.append(get_pixel(img, center, x-1, y + 1)) 
      
    # right 
    val_ar.append(get_pixel(img, center, x, y + 1)) 
      
    # bottom_right 
    val_ar.append(get_pixel(img, center, x + 1, y + 1)) 
      
    # bottom 
    val_ar.append(get_pixel(img, center, x + 1, y)) 
      
    # bottom_left 
    val_ar.append(get_pixel(img, center, x + 1, y-1)) 
      
    # left 
    val_ar.append(get_pixel(img, center, x, y-1)) 
       
    # Now, we need to convert binary 
    # values to decimal 
    power_val = [1, 2, 4, 8, 16, 32, 64, 128] 
   
    val = 0
      
    for i in range(len(val_ar)): 
        val += val_ar[i] * power_val[i] 
          
    return val

In [3]:
# determine extension of images
## brain tumor positive images
path = 'brain_tumor_dataset/brain_mri/yes'
# path = 'brain_tumor_dataset/otak/yes'
ext_set = set()
for child in Path(path).iterdir():
    ext = Path(child).suffix
    ext_set.add(ext)
print(f'positive image extensions: {ext_set}')

## brain tumor negative images
path = 'brain_tumor_dataset/brain_mri/no'
# path = 'brain_tumor_dataset/otak/no'

ext_set = set()
for child in Path(path).iterdir():
    ext = Path(child).suffix
    ext_set.add(ext)
print(f'negative image extensions: {ext_set}')

positive image extensions: {'.jpg'}
negative image extensions: {'.jpg'}


In [4]:
## postive
path = 'brain_tumor_dataset/brain_mri/yes'
# path = 'brain_tumor_dataset/otak/yes'

pos_li = list(Path(path).glob('*.jpg'))

## negative
path = 'brain_tumor_dataset/brain_mri/no'
# path = 'brain_tumor_dataset/otak/no'

neg_li = list(Path(path).glob('*.jpg'))
# neg_li.extend(list(Path(path).glob('*.jpg')))

# display number of images
print(f'Brain Tumor Positive images: {len(pos_li)}\nBrain Tumor Negative images: {len(neg_li)}')

Brain Tumor Positive images: 555
Brain Tumor Negative images: 335


In [11]:
# imaa = cv2.imread(str(pos_li[0]))
# gray = cv2.cvtColor(imaa,cv2.COLOR_BGR2RGB)
# cv2.imshow("Sda",imaa)
# cv2.waitKey(0) # waits until a key is pressed
# cv2.destroyAllWindows() # destroys the window showing image
# print(gray)
# imaa.shape

In [5]:
# create numpy array placeholder for pixels with 1 channel (grey scale)
IMG_SIZE = 300
pos_data = np.empty((len(pos_li), IMG_SIZE, IMG_SIZE, 1), dtype=np.float32)
neg_data = np.empty((len(neg_li), IMG_SIZE, IMG_SIZE, 1), dtype=np.float32)
# ^ float data type must be used to save precise pixel values

In [6]:
# convert images to numpy arrays
## positive
for i, img_path in enumerate(sorted(pos_li)):
    # load image
    img = cv2.imread(str(img_path))
    # convert BGR to RGB (since CV2 reads in BGR)
    img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
    # resize image with 1 channel
    img = resize(img, output_shape=(IMG_SIZE, IMG_SIZE, 1), preserve_range=True)
    # save to x_data
    pos_data[i] = img
## negative
for i, img_path in enumerate(sorted(neg_li)):
    # load image
    img = cv2.imread(str(img_path))
    # convert BGR to RGB (since CV2 reads in BGR)
    img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
    # resize image with 1 channel
    img = resize(img, output_shape=(IMG_SIZE, IMG_SIZE, 1), preserve_range=True)
    # save to x_data
    neg_data[i] = img

In [7]:
print(pos_data[0])

[[[0.]
  [0.]
  [0.]
  ...
  [0.]
  [0.]
  [0.]]

 [[0.]
  [0.]
  [0.]
  ...
  [0.]
  [0.]
  [0.]]

 [[0.]
  [0.]
  [0.]
  ...
  [0.]
  [0.]
  [0.]]

 ...

 [[0.]
  [0.]
  [0.]
  ...
  [0.]
  [0.]
  [0.]]

 [[0.]
  [0.]
  [0.]
  ...
  [0.]
  [0.]
  [0.]]

 [[0.]
  [0.]
  [0.]
  ...
  [0.]
  [0.]
  [0.]]]


In [8]:
# scale image arrays
pos_data /= 255
neg_data /= 255

In [9]:
pos_data[0].max()

0.9506222

In [10]:
#make directory for LBP image
lbp = ['LBP_yes', 'LBP_no']
try:
    os.mkdir('brain_tumor_dataset/preprocess')
    os.mkdir('brain_tumor_dataset/preprocess/data_LBP/')
    for x in lbp:
      os.mkdir('brain_tumor_dataset/preprocess/data_LBP/'+x)
except OSError:
    # pass
    print('not run')


In [19]:
# # print(str(pos_li[0]))
# newPath = str(pos_li[0]).split('\\')
# # new
# # fileLoc = "data/LBP_COVID/"+newPath[2]

# newPath[2]

In [11]:
# LBP Positif
for i, img_path in enumerate(sorted(pos_li)):
    height, width, _ = pos_data[i].shape
    # Create a numpy array as  
    # the same height and width  
    # of RGB image 
    img_lbp = np.zeros((height, width), 
                       np.uint8) 
    
    for j in range(0, height): 
        for k in range(0, width): 
            img_lbp[j, k] = lbp_calculated_pixel(pos_data[i], j, k)
    
    newPath = str(pos_li[i]).split('\\')
    fileLoc = "brain_tumor_dataset/preprocess/data_LBP/LBP_yes/"+newPath[3]
    cv2.imwrite(fileLoc, img_lbp)

In [12]:
# LBP Negatif
for i, img_path in enumerate(sorted(neg_li)):
    height, width, _ = neg_data[i].shape
    # Create a numpy array as  
    # the same height and width  
    # of RGB image 
    img_lbp = np.zeros((height, width), 
                       np.uint8)
    
    for j in range(0, height): 
        for k in range(0, width): 
            img_lbp[j, k] = lbp_calculated_pixel(neg_data[i], j, k)
    
    newPath = str(neg_li[i]).split('\\')
    fileLoc = "brain_tumor_dataset/preprocess/data_LBP/LBP_no/"+newPath[3]
    cv2.imwrite(fileLoc, img_lbp)